# **Replication Models**

In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stat
from lmfit import Model

## Differential

### Reactive Happiness Model (RHM)

In [2]:
data = pd.read_csv('../Data/Replication_data_rescale.csv')

save = {'CR':[], 'EV':[], 'PE':[], 'H':[], 'Certain Reward':[], 'Expected Value':[], 'Prediction Error':[], 'Forgetting Factor':[], 'BIC':[], 'R^2':[]}

# fit each participant
for s in range(111, 162):

    if len(data[data['id'] == s]) > 0:

        print(s)

        df1 = data[data['id'] == s]

        CR_data = np.array(df1['certain reward'])[0:]
        EV_data = np.array(df1['expected value'])[0:]
        PE_data = np.array(df1['prediction error'])[0:]
        H_data = np.array(df1['happiness rating'])[0:]
        
        # calculate change in happiness time series
        H_cleaned = H_data[~np.isnan(H_data)]
        H_change = []
        for i in range(1, len(H_cleaned)):
             H_change.append(H_cleaned[i]-H_cleaned[i-1])
        valid = []
        for i, j in enumerate(H_data):
            if ~np.isnan(j):
                valid.append(int(i))

        m = len(H_change)
        # print(m)
        time = [i for i in range(0, m)]

        # model fitting function
        def happiness(CR_coefficient, EV_coefficient, PE_coefficient, F_factor, CR, EV, PE, Valid):
            predictions = np.zeros(m)
            lastCR = CR[0]
            lastEV = EV[0]
            lastPE = PE[0]

            # fit each time point
            for i in range(0,m):
                    k = Valid[i+1]
                    CR_sum = 0
                    EV_sum = 0
                    PE_sum = 0

                    # calculate value signaling time series
                    for j in range(0,int(k)+1):
                            CR_sum += pow(F_factor,(k-j)) * CR[j]
                            EV_sum += pow(F_factor,(k-j)) * EV[j]
                            PE_sum += pow(F_factor,(k-j)) * PE[j]
                    
                    # fitting function
                    predictions[i] = ((CR_sum-lastCR) * CR_coefficient + (EV_sum-lastEV) * EV_coefficient + (PE_sum-lastPE) * PE_coefficient)
                    lastCR = CR_sum
                    lastEV = EV_sum
                    lastPE = PE_sum
            return predictions

        gmodel = Model(happiness, independent_vars=['CR', 'EV', 'PE', 'Valid'])
        params = gmodel.make_params(CR_coefficient=0.05, EV_coefficient=0.05, PE_coefficient=0.1, F_factor=0.5)
        params['CR_coefficient'].set(min = -1, max = 1)
        params['EV_coefficient'].set(min = -1, max = 1)
        params['PE_coefficient'].set(min = -1, max = 1)
        params['F_factor'].set(min = 0, max = 1)
        result = gmodel.fit(H_change, params, CR = CR_data, EV=EV_data, PE=PE_data, Valid=valid)

        # output results
        save['CR'].append(CR_data)
        save['EV'].append(EV_data)
        save['PE'].append(PE_data)
        save['H'].append(H_data)
        save['Certain Reward'].append(result.params['CR_coefficient'].value)
        save['Expected Value'].append(result.params['EV_coefficient'].value)
        save['Prediction Error'].append(result.params['PE_coefficient'].value)
        save['Forgetting Factor'].append(result.params['F_factor'].value)
        save['BIC'].append(result.bic)
        residuals = result.residual
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((H_change - np.mean(H_change))**2)
        r_squared = 1 - (ss_res / ss_tot)
        print('R^2: ', r_squared)
        save['R^2'].append(r_squared)
save = pd.DataFrame(save)
save.to_csv('../Outputs/Lmfit_Replication_RHM_Dif.csv')

111
R^2:  0.22412799338844525
112
R^2:  0.48507610465449547
113
R^2:  0.5916392952540837
114
R^2:  0.7259969038892837
115
R^2:  0.7645661863576748
116
R^2:  0.3091242863554968
117
R^2:  0.23366609215939038
118
R^2:  0.602985553664531
119
R^2:  0.5276372823302438
120
R^2:  0.13896479879172474
121
R^2:  0.35321676189699924
122
R^2:  0.5985041103508993
123
R^2:  0.22749565972397912
124
R^2:  0.5229264328551138
125
R^2:  0.5387512564510895
126
R^2:  0.7877239379700886
127
R^2:  0.7273368972677767
128
R^2:  0.4596791087432732
129
R^2:  0.38197244329537927
130
R^2:  0.7838953565998668
131
R^2:  0.4461044735205516
132
R^2:  0.1730817011972804
133
R^2:  0.16823592899331297
134
R^2:  0.06842542799272566
135
R^2:  0.5195390700544454
136
R^2:  0.709527438882638
137
R^2:  0.4054965217931803
138
R^2:  0.5571558697687657
139
R^2:  0.19550552607445393
140
R^2:  0.33753869042917295
141
R^2:  0.01866740748897855
142
R^2:  0.500242049817107
143
R^2:  0.1252971740157931
144
R^2:  0.4243020981058446
145
R

### DynAffect-C

In [4]:
data = pd.read_csv('../Data/Replication_data_rescale.csv')

save = {'CR':[], 'EV':[], 'PE':[], 'H':[], 'Certain Reward':[], 'Expected Value':[], 'Prediction Error':[], 'Forgetting Factor':[], 'Baseline':[], 'Linear Attraction':[], 'Cubic Attraction':[],  'BIC':[], 'R^2':[]}

# fit each participant
for s in range(111, 162):

    if len(data[data['id'] == s] > 0):

        print(s)

        df1 = data[data['id'] == s]

        CR_data = np.array(df1['certain reward'])[0:]
        EV_data = np.array(df1['expected value'])[0:]
        PE_data = np.array(df1['prediction error'])[0:]
        H_data = np.array(df1['happiness rating'])[0:]
        
        # calculate change in happiness time series
        H_cleaned = H_data[~np.isnan(H_data)]
        H_change = []
        for i in range(1, len(H_cleaned)):
             H_change.append(H_cleaned[i]-H_cleaned[i-1])
        valid = []
        for i, j in enumerate(H_data):
            if ~np.isnan(j):
                valid.append(int(i))

        m = len(H_change)
        # print(m)
        time = [i for i in range(0, m)]

        # model fitting function
        def happiness(CR_coefficient, EV_coefficient, PE_coefficient, F_factor, Mu, Beta, Alpha, CR, EV, PE, Valid, H_cleaned):
            predictions = np.zeros(m)
            lastCR = CR[0]
            lastEV = EV[0]
            lastPE = PE[0]

            # fit each time point
            for i in range(0,m):
                    k = Valid[i+1]
                    CR_sum = 0
                    EV_sum = 0
                    PE_sum = 0

                    # calculate value signaling time series
                    for j in range(0,int(k)+1):
                            CR_sum += pow(F_factor,(k-j)) * CR[j]
                            EV_sum += pow(F_factor,(k-j)) * EV[j]
                            PE_sum += pow(F_factor,(k-j)) * PE[j]
                    
                    # fitting function
                    predictions[i] = (Mu - H_cleaned[i]) * Beta + pow((Mu - H_cleaned[i]), 3) * Alpha + ((CR_sum-lastCR) * CR_coefficient + (EV_sum-lastEV) * EV_coefficient + (PE_sum-lastPE) * PE_coefficient)
                    lastCR = CR_sum
                    lastEV = EV_sum
                    lastPE = PE_sum
            return predictions

        gmodel = Model(happiness, independent_vars=['CR', 'EV', 'PE', 'Valid', 'H_cleaned'])
        params = gmodel.make_params(CR_coefficient=0.05, EV_coefficient=0.05, PE_coefficient=0.1, Mu = 50, F_factor=0.5, Beta = 0.5, Alpha = 0)
        params['CR_coefficient'].set(min = -1, max = 1)
        params['EV_coefficient'].set(min = -1, max = 1)
        params['PE_coefficient'].set(min = -1, max = 1)
        params['F_factor'].set(min = 0, max = 1)
        params['Mu'].set(min = 1, max = 99)
        params['Beta'].set(min = -5, max = 5)
        params['Alpha'].set(min = 0, max = 1)
        result = gmodel.fit(H_change, params, CR = CR_data, EV=EV_data, PE=PE_data, Valid=valid, H_cleaned = H_cleaned)

        # output results
        save['CR'].append(CR_data)
        save['EV'].append(EV_data)
        save['PE'].append(PE_data)
        save['H'].append(H_data)
        save['Certain Reward'].append(result.params['CR_coefficient'].value)
        save['Expected Value'].append(result.params['EV_coefficient'].value)
        save['Prediction Error'].append(result.params['PE_coefficient'].value)
        save['Forgetting Factor'].append(result.params['F_factor'].value)
        save['Baseline'].append(result.params['Mu'].value)
        save['Linear Attraction'].append(result.params['Beta'].value)
        save['Cubic Attraction'].append(result.params['Alpha'].value)
        save['BIC'].append(result.bic)
        residuals = result.residual
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((H_change - np.mean(H_change))**2)
        r_squared = 1 - (ss_res / ss_tot)
        print('R^2: ', r_squared)
        save['R^2'].append(r_squared)
save = pd.DataFrame(save)
save.to_csv('../Outputs/Lmfit_Replication_DC_Dif.csv')

111
R^2:  0.46469898664062503
112
R^2:  0.7681796413632362
113
R^2:  0.6436440531332902
114
R^2:  0.7731786943880685
115
R^2:  0.7808323839177398
116
R^2:  0.45894100139055316
117
R^2:  0.5630196916021492
118
R^2:  0.6268206032267907
119
R^2:  0.7092693774846075
120
R^2:  0.2602773611498401
121
R^2:  0.4455469634434851
122
R^2:  0.6998022818168665
123
R^2:  0.32615155996434053
124
R^2:  0.5708291878159355
125
R^2:  0.6675755103146215
126
R^2:  0.8236958863611524
127
R^2:  0.7779350531894857
128
R^2:  0.5535925331854605
129
R^2:  0.5311183114022373
130
R^2:  0.8602063548796086
131
R^2:  0.5229003502140797
132
R^2:  0.2672106077778431
133
R^2:  0.7743857184241018
134
R^2:  0.4175930805318757
135
R^2:  0.6718407555118314
136
R^2:  0.749888804103217
137
R^2:  0.5011640792170431
138
R^2:  0.5906440670364312
139
R^2:  0.36490968355310593
140
R^2:  0.3933375017992571
141
R^2:  0.1942029008090913
142
R^2:  0.6844199733100576
143
R^2:  0.5121076135274618
144
R^2:  0.553282061292474
145
R^2:  0.

## Non-Differential

### RHM

In [6]:
data = pd.read_csv('../Data/Replication_data_rescale.csv')

save = {'CR':[], 'EV':[], 'PE':[], 'H':[], 'Certain Reward':[], 'Expected Value':[], 'Prediction Error':[], 'Forgetting Factor':[], 'Baseline':[],'BIC':[], 'R^2':[]}

# fit each participant
for s in range(111, 162):

    if len(data[data['id'] == s]) > 0:

        print(s)

        df1 = data[data['id'] == s]

        CR_data = np.array(df1['certain reward'])[0:]
        EV_data = np.array(df1['expected value'])[0:]
        PE_data = np.array(df1['prediction error'])[0:]
        H_data = np.array(df1['happiness rating'])[0:]
        
        # remove first happiness value
        H_cleaned = H_data[~np.isnan(H_data)]
        H_cleaned = H_cleaned[1:]
        m = len(H_cleaned)
        # print(m)
        valid = []
        for i, j in enumerate(H_data):
            if ~np.isnan(j):
                valid.append(int(i))
        # print(valid)
        # print(H_cleaned)

        time = [i for i in range(0, m)]

        # model fitting function
        def happiness(CR_coefficient, EV_coefficient, PE_coefficient, F_factor, Mu, CR, EV, PE, Valid):
            predictions = np.zeros(m)

            # fit each time point
            for i in range(0,m):
                    k = Valid[i+1]
                    CR_sum = 0
                    EV_sum = 0
                    PE_sum = 0

                    # calculate value signaling time series
                    for j in range(0,int(k)+1):
                        CR_sum += pow(F_factor,(k-j)) * CR[j]
                        EV_sum += pow(F_factor,(k-j)) * EV[j]
                        PE_sum += pow(F_factor,(k-j)) * PE[j]
                    
                    # fitting function
                    predictions[i] = Mu + (CR_sum * CR_coefficient + EV_sum * EV_coefficient + PE_sum * PE_coefficient)
            return predictions

        gmodel = Model(happiness, independent_vars=['CR', 'EV', 'PE', 'Valid'])
        params = gmodel.make_params(CR_coefficient=0.05, EV_coefficient=0.05, PE_coefficient=0.1, Base = 50, F_factor=0.5)
        params['CR_coefficient'].set(min = -1, max = 1)
        params['EV_coefficient'].set(min = -1, max = 1)
        params['PE_coefficient'].set(min = -1, max = 1)
        params['F_factor'].set(min = 0, max = 1)
        params['Mu'].set(min = 1, max = 99)
        result = gmodel.fit(H_cleaned, params, CR = CR_data, EV=EV_data, PE=PE_data, Valid=valid)

        # output results
        save['CR'].append(CR_data)
        save['EV'].append(EV_data)
        save['PE'].append(PE_data)
        save['H'].append(H_data)
        save['Certain Reward'].append(result.params['CR_coefficient'].value)
        save['Expected Value'].append(result.params['EV_coefficient'].value)
        save['Prediction Error'].append(result.params['PE_coefficient'].value)
        save['Forgetting Factor'].append(result.params['F_factor'].value)
        save['Baseline'].append(result.params['Mu'].value)
        save['BIC'].append(result.bic)
        residuals = result.residual
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((H_cleaned - np.mean(H_cleaned))**2)
        r_squared = 1 - (ss_res / ss_tot)
        print('R^2: ', r_squared)
        save['R^2'].append(r_squared)
save = pd.DataFrame(save)
save.to_csv('../Outputs/Lmfit_Replication_RHM_NonDif.csv')

111
R^2:  0.3294842273644175
112
R^2:  0.4978617212237645
113
R^2:  0.6058158281931527
114
R^2:  0.5465079193878792
115
R^2:  0.693285712484814
116
R^2:  0.5967269870380173
117
R^2:  0.18475550515478978
118
R^2:  0.8262779007663358
119
R^2:  0.732083647501504
120
R^2:  0.5281460923112051
121
R^2:  -2.925217361703706
122
R^2:  0.7532932150889284
123
R^2:  0.2878952011566256
124
R^2:  0.6382953831601936
125
R^2:  0.8817870103941832
126
R^2:  0.754093646343116
127
R^2:  0.6921685231493155
128
R^2:  0.37212253190394473
129
R^2:  0.3853317839419891
130
R^2:  0.7793519372419621
131
R^2:  0.8003601196620447
132
R^2:  0.7816679594175382
133
R^2:  0.179938549437216
134
R^2:  0.2347597325546914
135
R^2:  0.5462727104480263
136
R^2:  0.614622733440205
137
R^2:  0.6544387818806519
138
R^2:  0.7875479837767854
139
R^2:  0.6439000835771527
140
R^2:  0.661010740404784
141
R^2:  0.3801948228058377
142
R^2:  0.6284788391239245
143
R^2:  0.24553189600989633
144
R^2:  0.5308550952174247
145
R^2:  0.41966

### DynAffect-C

In [7]:
data = pd.read_csv('../Data/Replication_data_rescale.csv')

save = {'CR':[], 'EV':[], 'PE':[], 'H':[], 'Certain Reward':[], 'Expected Value':[], 'Prediction Error':[], 'Forgetting Factor':[], 'Baseline':[], 'Linear Attraction':[], 'Cubic Attraction':[],  'BIC':[], 'R^2':[]}

# fit each participant
for s in range(111, 162):

    if len(data[data['id'] == s] > 0):

        print(s)

        df1 = data[data['id'] == s]

        CR_data = np.array(df1['certain reward'])[0:]
        EV_data = np.array(df1['expected value'])[0:]
        PE_data = np.array(df1['prediction error'])[0:]
        H_data = np.array(df1['happiness rating'])[0:]
        

        # remove first happiness value
        H_cleaned = H_data[~np.isnan(H_data)]
        H_cleaned2 = H_cleaned[1:]
        m = len(H_cleaned2)
        # print(m)
        valid = []
        for i, j in enumerate(H_data):
            if ~np.isnan(j):
                valid.append(int(i))
        # print(valid)
        # print(H_cleaned)

        time = [i for i in range(0, m)]

        # model fitting function
        def happiness(CR_coefficient, EV_coefficient, PE_coefficient, Mu, F_factor, Beta, Alpha, CR, EV, PE, Valid, H_cleaned):
            predictions = np.zeros(m)

            # fit each time point
            for i in range(0,m):
                    k = Valid[i+1]
                    CR_sum = 0
                    EV_sum = 0
                    PE_sum = 0

                    # calculate value signaling time series
                    for j in range(0,int(k)+1):
                        CR_sum += pow(F_factor,(k-j)) * CR[j]
                        EV_sum += pow(F_factor,(k-j)) * EV[j]
                        PE_sum += pow(F_factor,(k-j)) * PE[j]
                    
                    # fitting function
                    predictions[i] = Mu - ((Mu - H_cleaned[i]) * np.exp(-1*Beta))/np.sqrt(1+(Alpha/Beta)*np.power(Mu-H_cleaned[i],2)*(1-np.exp(-2*Beta))) + (CR_sum * CR_coefficient + EV_sum * EV_coefficient + PE_sum * PE_coefficient)
            return predictions

        gmodel = Model(happiness, independent_vars=['CR', 'EV', 'PE', 'Valid', 'H_cleaned'])
        params = gmodel.make_params(CR_coefficient=0.05, EV_coefficient=0.05, PE_coefficient=0.1, Mu = 50, F_factor=0.5, Beta = 0.5, Alpha = 0)
        params['CR_coefficient'].set(min = -1, max = 1)
        params['EV_coefficient'].set(min = -1, max = 1)
        params['PE_coefficient'].set(min = -1, max = 1)
        params['F_factor'].set(min = 0, max = 1)
        params['Mu'].set(min = 1, max = 99)
        params['Beta'].set(min = -5, max = 5)
        params['Alpha'].set(min = 0, max = 1)
        result = gmodel.fit(H_cleaned2, params, CR = CR_data, EV=EV_data, PE=PE_data, Valid=valid, H_cleaned = H_cleaned)

        # output results
        save['CR'].append(CR_data)
        save['EV'].append(EV_data)
        save['PE'].append(PE_data)
        save['H'].append(H_data)
        save['Certain Reward'].append(result.params['CR_coefficient'].value)
        save['Expected Value'].append(result.params['EV_coefficient'].value)
        save['Prediction Error'].append(result.params['PE_coefficient'].value)
        save['Forgetting Factor'].append(result.params['F_factor'].value)
        save['Baseline'].append(result.params['Mu'].value)
        save['Linear Attraction'].append(result.params['Beta'].value)
        save['Cubic Attraction'].append(result.params['Alpha'].value)
        save['BIC'].append(result.bic)
        residuals = result.residual
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((H_cleaned2 - np.mean(H_cleaned2))**2)
        r_squared = 1 - (ss_res / ss_tot)
        print('R^2: ', r_squared)
        save['R^2'].append(r_squared)
save = pd.DataFrame(save)
save.to_csv('../Outputs/Lmfit_Replication_DC_NonDif.csv')

111
R^2:  0.3595793895957631
112
R^2:  0.49738519379260915
113
R^2:  0.6647565797221495
114
R^2:  0.6748166736817498
115
R^2:  0.805307543759225
116
R^2:  0.6070225294898189
117
R^2:  0.18461688990038583
118
R^2:  0.7951969855790824
119
R^2:  0.7390692976891624
120
R^2:  0.6459275870400516
121
R^2:  0.6322519999095954
122
R^2:  0.6612846538989376
123
R^2:  0.7521234108703949
124
R^2:  0.6849945260617795
125
R^2:  0.9366421083565671
126
R^2:  0.8083348265122831
127
R^2:  0.7191464914722623
128
R^2:  0.5091956678817238
129
R^2:  0.5115444532570746
130
R^2:  0.7792037605328233
131
R^2:  0.7499823417473974
132
R^2:  0.6720750933899129
133
R^2:  0.17978711251077506
134
R^2:  0.24198477995172085
135
R^2:  0.546008961090394
136
R^2:  0.6907475292687437
137
R^2:  0.6319976996736669
138
R^2:  0.8762183042491409
139
R^2:  0.6355255884755238
140
R^2:  0.7449512587877689
141
R^2:  0.4724328923488965
142
R^2:  0.6556081690677917
143
R^2:  0.2452190083932817
144
R^2:  0.7549862168453033
145
R^2:  0.